In [108]:
import pandas as pd
import bs4
import json
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [64]:
def create_pipeline(vectorizer, transformer, classifier):
    return Pipeline(
        [("vectorizer", vectorizer),
         ("transformer", transformer),
         ("classifier", classifier)
         ]
    )

### Посмотрим на тестовые данные и подготовим их

In [15]:
data = bs4.BeautifulSoup(open("data/test.csv", encoding="utf-8"))
for a in data.find_all("review"):
    print(a.text)
    print("-----------------")

Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.

-----------------
ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если 
постоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.
 Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсо

In [17]:
test = [line.text for line in data.find_all("review")]
test[:1]

['Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.\nНесколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.\n']

### Нужны данные по телефонам на русском языке. Возьмем спарсенные данные

In [48]:
data = pd.read_json('data/train.json', orient='records', lines=True, encoding="utf-8")

In [49]:
data.head()

,text,rating
0,"Достоинства: 1) отличный дизайн, мой синий кор...",4
1,"Достоинства: Качество связи,сборки.Удобство qw...",5
2,Достоинства: Могу сказать сразу. Телефон шикар...,5
3,Достоинства: -качественная сборка -QWERTY-клав...,5
4,Достоинства: Купил вчера. во-первых это полная...,5


In [105]:
# Почтистим данные, чтоб были похожи на данные в тестовой выборке

data.text = data.text.str.replace("Достоинства:", "").replace("Недостатки:", "")

**Т.к. оценок с рейтингом 5 больше всего, для начала только их возьмем за положительный отзыв, все остальные за отрицательный.**

In [75]:
data["is_five"] = (data.rating == 5).astype(int)

In [107]:
data.head(2)

,text,rating,is_five
0,"1) отличный дизайн, мой синий корпус смотритя...",4,0
1,"Качество связи,сборки.Удобство qwerty клавиат...",5,1


In [94]:
data.is_five.value_counts()

1    3324
0    1681
Name: rating, dtype: int64

In [110]:
pipe = create_pipeline(
    CountVectorizer(),
    TfidfTransformer(),
    CalibratedClassifierCV()
)

In [111]:
res = cross_val_score(pipe, data.text, data.is_five, cv=5, scoring="accuracy")

In [113]:
res.mean(), res.mean() > 0.85

(0.7748236825051196, False)

**CV результат не удовлитворительный, но попробуем отправить результат на каггл.**

In [136]:
pipe.fit(data.text, data.is_five);

In [161]:
y_res = pipe.predict(test)
y_res

array([0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1])

In [163]:
y_res = y_res.astype(str)
y_res[y_res == "0"] = "neg"
y_res[y_res == "1"] = "pos"
pd.DataFrame({"Id": range(100), "y": y_res}).to_csv("data/res.csv", index=False)

<img src="data/simple_model.PNG">

### Как видно такой простой модели хватило для нужной точности > 0.85